In [1]:
import os
from tasrif.processing_pipeline import SequenceOperator, ComposeOperator, MapIterableOperator
from tasrif.data_readers.fitbit_interday_dataset import FitbitInterdayDataset
from tasrif.processing_pipeline.pandas import ConvertToDatetimeOperator, SetIndexOperator, ConcatOperator, MergeOperator, \
AsTypeOperator, DropFeaturesOperator
from tasrif.processing_pipeline.custom import CreateFeatureOperator, AggregateOperator, FlattenOperator

In [21]:
interday_folder_path = os.environ.get('FITBIT_INTERDAY_PATH', '/mnt/data/fitbit-data/')
interday_folder_path

'/Users/ahmad/mnt/data/fitbit-data/'

In [4]:
import tasrif.yaml_parser as yaml_parser
import yaml    

In [22]:
%env FITBIT_INTERDAY_PATH

'/Users/ahmad/mnt/data/fitbit-data/'

In [23]:
with open("example.yaml", "r") as stream:
    try:
        p = yaml_parser.from_yaml(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [24]:
df = p.process()
df

[            Calories Burned  Steps  Distance  Minutes Sedentary  \
 Date                                                              
 2019-06-01             3406  13007      9.12                288   
 2019-06-02             3541  13854      9.66                803   
 2019-06-03             3866  15032     10.48                830   
 2019-06-04             2896   7136      4.97                449   
 2019-06-05             2838   7669      5.35                  0   
 2019-06-06             3121   9451      6.59                109   
 2019-06-07             3269   9271      6.46                 47   
 2019-06-08             3960  14766     10.29                  0   
 2019-06-09             3407  11197      7.80                359   
 2019-06-10             3192   9395      6.55                161   
 2019-06-11             3444  10538      7.34                204   
 2019-06-12             4176  15100     10.52                472   
 2019-06-13             4048  15731     10.96   